# Disable Organization-Wide Access for Databricks Apps

This notebook:
1. Lists all apps with "anyone in my organization can use" enabled
2. Disables org-wide access for each app (changes to "only people with access can use")

**Prerequisites:**
- You need `CAN_MANAGE` permission on apps to modify their permissions
- Databricks SDK should be installed: `%pip install databricks-sdk`


## Step 1: Install Dependencies (if needed)


In [0]:
%pip install databricks-sdk==0.49.0 --quiet
dbutils.library.restartPython()


## Step 2: Import Libraries and Initialize Client


In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.apps import AppAccessControlRequest, AppPermissionLevel

# Initialize the Databricks WorkspaceClient
# In Databricks notebooks, this automatically uses the notebook's context
w = WorkspaceClient()

print("✓ WorkspaceClient initialized successfully")


## Step 3: Define Helper Functions


In [0]:
def is_org_wide_enabled(acl_entry) -> bool:
    """Check if an ACL entry represents organization-wide access."""
    grp = (getattr(acl_entry, "group_name", None) or "").lower()
    if grp == 'account users':
        permission_levels = getattr(acl_entry, "all_permissions", [])
        for permission_level in permission_levels:
            lvl = (getattr(permission_level, "permission_level", "") or "").value.upper()
            if lvl == 'CAN_USE':
                return True
    return False


def get_all_org_wide_apps(w: WorkspaceClient) -> list:
    """Get all apps that have 'anyone in my organization can use' enabled."""
    shared = []
    
    for app in w.apps.list():
        try:
            perms = w.apps.get_permissions(app.name)
            acl = getattr(perms, "access_control_list", []) or []
            
            if any(is_org_wide_enabled(entry) for entry in acl):
                shared.append({
                    "name": app.name,
                    "status": getattr(app, "status", None),
                    "description": getattr(app, "description", None),
                    "owner": getattr(app, "creator", None) or getattr(app, "owner", None)
                })
        except Exception as e:
            print(f"⚠️  Could not check app '{app.name}': {e}")
            continue
    
    return shared


def disable_org_wide_access(w: WorkspaceClient, app_name: str) -> bool:
    """Disable 'anyone in my organization can use' permission for an app."""
    try:
        perms = w.apps.get_permissions(app_name)
        acl = getattr(perms, "access_control_list", []) or []
        
        # Build new ACL excluding "account users"
        new_acl = []
        found_account_users = False
        
        for entry in acl:
            grp = (getattr(entry, "group_name", None) or "").lower()
            
            if grp == 'account users':
                found_account_users = True
                continue
            
            # Keep all other entries
            permission_levels = getattr(entry, "all_permissions", [])
            for perm in permission_levels:
                perm_level = getattr(perm, "permission_level", None)
                if perm_level:
                    new_acl.append(AppAccessControlRequest(
                        user_name=getattr(entry, "user_name", None),
                        group_name=getattr(entry, "group_name", None),
                        service_principal_name=getattr(entry, "service_principal_name", None),
                        permission_level=perm_level
                    ))
                    break
        
        if not found_account_users:
            return True  # Already restricted
        
        # Convert to API request format
        body = {"access_control_list": []}
        for entry in new_acl:
            acl_entry = {}
            if entry.user_name:
                acl_entry["user_name"] = entry.user_name
            if entry.group_name:
                acl_entry["group_name"] = entry.group_name
            if entry.service_principal_name:
                acl_entry["service_principal_name"] = entry.service_principal_name
            if entry.permission_level:
                perm_str = str(entry.permission_level.value) if hasattr(entry.permission_level, 'value') else str(entry.permission_level)
                acl_entry["permission_level"] = perm_str
            body["access_control_list"].append(acl_entry)
        
        # Update permissions
        w.api_client.do('PUT', f'/api/2.0/permissions/apps/{app_name}', body=body)
        return True
        
    except Exception as e:
        print(f"   Error: {e}")
        return False


print("✓ Helper functions defined")


## Step 4: Find All Apps with Organization-Wide Access

This will scan all apps in the workspace and identify which ones have "anyone in my organization can use" enabled.


In [0]:
print("🔍 Scanning for apps with organization-wide access...\n")

org_wide_apps = get_all_org_wide_apps(w)

if not org_wide_apps:
    print("✓ No apps found with 'anyone in my organization can use' enabled.")
    print("  Nothing to change!")
else:
    print(f"📋 Found {len(org_wide_apps)} app(s) with organization-wide access:\n")
    print("=" * 80)
    
    for i, app in enumerate(org_wide_apps, 1):
        print(f"{i}. {app['name']}")
        if app.get('owner'):
            print(f"   Owner: {app['owner']}")
        if app.get('status'):
            print(f"   Status: {app['status']}")
        if app.get('description'):
            print(f"   Description: {app['description']}")
        print()
    
    print("=" * 80)


## Step 5: Review and Confirm

⚠️ **Important:** This will change permissions for all apps listed above.

After disabling org-wide access:
- Only users with explicit permissions will be able to access each app
- Users who relied on "account users" group access will lose access

**Set `CONFIRM_DISABLE = True` to proceed with the changes.**


In [0]:
# Set this to True to proceed with disabling org-wide access
CONFIRM_DISABLE = False

if not CONFIRM_DISABLE:
    print("⚠️  CONFIRM_DISABLE is set to False")
    print("   Review the apps above and set CONFIRM_DISABLE = True to proceed.")
else:
    print("✓ Confirmation received. Proceeding with permission changes...")


## Step 6: Disable Organization-Wide Access for All Apps

This will remove "account users" from each app's ACL, changing the setting to "only people with access can use".


In [0]:
if not org_wide_apps:
    print("Nothing to do - no apps with org-wide access found.")
elif not CONFIRM_DISABLE:
    print("⚠️  Skipping permission changes - CONFIRM_DISABLE is not True")
else:
    print(f"🔄 Disabling organization-wide access for {len(org_wide_apps)} app(s)...\n")
    print("=" * 80)
    
    success_count = 0
    failed_count = 0
    failed_apps = []
    
    for i, app in enumerate(org_wide_apps, 1):
        app_name = app['name']
        print(f"\n{i}/{len(org_wide_apps)} Processing: {app_name}")
        
        success = disable_org_wide_access(w, app_name)
        
        if success:
            print(f"   ✓ Successfully changed to 'only people with access can use'")
            success_count += 1
        else:
            print(f"   ✗ Failed to change permission")
            failed_count += 1
            failed_apps.append(app_name)
    
    # Summary
    print("\n" + "=" * 80)
    print("\n📊 SUMMARY:")
    print(f"   Total apps processed: {len(org_wide_apps)}")
    print(f"   ✓ Successfully changed: {success_count}")
    print(f"   ✗ Failed: {failed_count}")
    
    if failed_apps:
        print("\n   Failed apps:")
        for app_name in failed_apps:
            print(f"   - {app_name}")
    
    print("\n" + "=" * 80)


## Step 7: Verify the Changes

Let's verify that organization-wide access has been disabled for all apps.


In [0]:
if CONFIRM_DISABLE and org_wide_apps:
    print("🔍 Verifying changes...\n")
    
    still_enabled = get_all_org_wide_apps(w)
    
    if not still_enabled:
        print("✅ SUCCESS! No apps have organization-wide access enabled anymore.")
    else:
        print(f"⚠️  Warning: {len(still_enabled)} app(s) still have org-wide access:")
        for app in still_enabled:
            print(f"   - {app['name']}")
else:
    print("Skipping verification - no changes were made.")


## ✅ Workflow Complete!

This notebook has completed the following steps:

1. ✅ Scanned all apps in the workspace
2. ✅ Identified apps with "anyone in my organization can use" enabled
3. ✅ Disabled org-wide access for each app (if `CONFIRM_DISABLE = True`)
4. ✅ Verified the changes

### What Changed:

- Apps no longer have "account users" group in their ACL
- Setting changed from "anyone in my organization can use" to "only people with access can use"
- Only users with explicit permissions can now access these apps

### What was Preserved:

- All other user, group, and service principal permissions remain unchanged
- App functionality is unaffected (only access control changed)


## Optional: Revert Changes (Re-enable Organization-Wide Access)

If you need to revert the changes and re-enable "anyone in my organization can use" for specific apps, use this section.

⚠️ **Use carefully** - This will re-add "account users" group with CAN_USE permission.


In [0]:
def enable_org_wide_access(w: WorkspaceClient, app_name: str) -> bool:
    """Enable 'anyone in my organization can use' permission for an app."""
    try:
        perms = w.apps.get_permissions(app_name)
        acl = getattr(perms, "access_control_list", []) or []
        
        # Check if already enabled
        if any(is_org_wide_enabled(entry) for entry in acl):
            return True  # Already enabled
        
        # Build new ACL with "account users" added
        new_acl = []
        
        for entry in acl:
            grp = (getattr(entry, "group_name", None) or "").lower()
            if grp == 'account users':
                continue  # Skip to avoid duplicates
            
            permission_levels = getattr(entry, "all_permissions", [])
            for perm in permission_levels:
                perm_level = getattr(perm, "permission_level", None)
                if perm_level:
                    new_acl.append(AppAccessControlRequest(
                        user_name=getattr(entry, "user_name", None),
                        group_name=getattr(entry, "group_name", None),
                        service_principal_name=getattr(entry, "service_principal_name", None),
                        permission_level=perm_level
                    ))
                    break
        
        # Add "account users" with CAN_USE
        new_acl.append(AppAccessControlRequest(
            group_name="account users",
            permission_level=AppPermissionLevel.CAN_USE
        ))
        
        # Convert to API format and update
        body = {"access_control_list": []}
        for entry in new_acl:
            acl_entry = {}
            if entry.user_name:
                acl_entry["user_name"] = entry.user_name
            if entry.group_name:
                acl_entry["group_name"] = entry.group_name
            if entry.service_principal_name:
                acl_entry["service_principal_name"] = entry.service_principal_name
            if entry.permission_level:
                perm_str = str(entry.permission_level.value) if hasattr(entry.permission_level, 'value') else str(entry.permission_level)
                acl_entry["permission_level"] = perm_str
            body["access_control_list"].append(acl_entry)
        
        w.api_client.do('PUT', f'/api/2.0/permissions/apps/{app_name}', body=body)
        return True
        
    except Exception as e:
        print(f"   Error: {e}")
        return False


print("✓ Revert function defined")


### Option A: Revert for Specific Apps

Provide a list of app names to re-enable organization-wide access for specific apps only.


In [0]:
# List specific app names to revert (re-enable org-wide access)
APPS_TO_REVERT = [
    # "app-name-1",
    # "app-name-2",
]

# Safety flag - must be set to True
CONFIRM_REVERT = False

if not APPS_TO_REVERT:
    print("No apps specified in APPS_TO_REVERT list")
elif not CONFIRM_REVERT:
    print("⚠️  CONFIRM_REVERT is set to False")
    print("   Set CONFIRM_REVERT = True to proceed with reverting changes")
else:
    print(f"🔄 Re-enabling organization-wide access for {len(APPS_TO_REVERT)} app(s)...\n")
    print("=" * 80)
    
    success_count = 0
    failed_count = 0
    already_enabled_count = 0
    
    for i, app_name in enumerate(APPS_TO_REVERT, 1):
        print(f"\n{i}/{len(APPS_TO_REVERT)} Processing: {app_name}")
        
        # Check if already enabled
        try:
            perms = w.apps.get_permissions(app_name)
            acl = getattr(perms, "access_control_list", []) or []
            
            if any(is_org_wide_enabled(entry) for entry in acl):
                print(f"   ℹ️  Already enabled - skipping")
                already_enabled_count += 1
                continue
        except Exception as e:
            print(f"   ✗ Could not check app: {e}")
            failed_count += 1
            continue
        
        # Enable org-wide access
        success = enable_org_wide_access(w, app_name)
        
        if success:
            print(f"   ✓ Successfully re-enabled 'anyone in my organization can use'")
            success_count += 1
        else:
            print(f"   ✗ Failed to enable permission")
            failed_count += 1
    
    # Summary
    print("\n" + "=" * 80)
    print("\n📊 REVERT SUMMARY:")
    print(f"   Total apps to revert: {len(APPS_TO_REVERT)}")
    print(f"   ✓ Successfully re-enabled: {success_count}")
    print(f"   ℹ️  Already enabled: {already_enabled_count}")
    print(f"   ✗ Failed: {failed_count}")
    print("\n" + "=" * 80)


### Option B: Revert All Previously Disabled Apps

Re-enable organization-wide access for all apps that were disabled in Step 6.

⚠️ **This will revert ALL changes made earlier in this notebook.**


In [0]:
# Safety flag - must be set to True
CONFIRM_REVERT_ALL = False

# Check if we have the list of apps that were disabled
if 'org_wide_apps' not in locals() or not org_wide_apps:
    print("⚠️  No apps to revert")
    print("   The variable 'org_wide_apps' is not available.")
    print("   Either re-run Step 4 to get the list, or use Option A above to specify apps manually.")
elif not CONFIRM_DISABLE:
    print("⚠️  Cannot revert - apps were not actually disabled in Step 6")
    print("   CONFIRM_DISABLE was False, so no changes were made.")
elif not CONFIRM_REVERT_ALL:
    print(f"⚠️  CONFIRM_REVERT_ALL is set to False")
    print(f"   This will re-enable org-wide access for {len(org_wide_apps)} app(s):")
    for app in org_wide_apps:
        print(f"     - {app['name']}")
    print()
    print("   Set CONFIRM_REVERT_ALL = True to proceed with reverting ALL changes")
else:
    print(f"🔄 Re-enabling organization-wide access for {len(org_wide_apps)} app(s)...\n")
    print("=" * 80)
    
    success_count = 0
    failed_count = 0
    already_enabled_count = 0
    
    for i, app in enumerate(org_wide_apps, 1):
        app_name = app['name']
        print(f"\n{i}/{len(org_wide_apps)} Processing: {app_name}")
        
        # Check if already enabled
        try:
            perms = w.apps.get_permissions(app_name)
            acl = getattr(perms, "access_control_list", []) or []
            
            if any(is_org_wide_enabled(entry) for entry in acl):
                print(f"   ℹ️  Already enabled - skipping")
                already_enabled_count += 1
                continue
        except Exception as e:
            print(f"   ✗ Could not check app: {e}")
            failed_count += 1
            continue
        
        # Enable org-wide access
        success = enable_org_wide_access(w, app_name)
        
        if success:
            print(f"   ✓ Successfully re-enabled 'anyone in my organization can use'")
            success_count += 1
        else:
            print(f"   ✗ Failed to enable permission")
            failed_count += 1
    
    # Summary
    print("\n" + "=" * 80)
    print("\n📊 REVERT SUMMARY:")
    print(f"   Total apps to revert: {len(org_wide_apps)}")
    print(f"   ✓ Successfully re-enabled: {success_count}")
    print(f"   ℹ️  Already enabled: {already_enabled_count}")
    print(f"   ✗ Failed: {failed_count}")
    print("\n" + "=" * 80)
    
    # Verify
    print("\n🔍 Verifying revert...\n")
    still_disabled = []
    for app in org_wide_apps:
        try:
            perms = w.apps.get_permissions(app['name'])
            acl = getattr(perms, "access_control_list", []) or []
            if not any(is_org_wide_enabled(entry) for entry in acl):
                still_disabled.append(app['name'])
        except:
            pass
    
    if not still_disabled:
        print("✅ SUCCESS! All apps have been reverted to organization-wide access.")
    else:
        print(f"⚠️  Warning: {len(still_disabled)} app(s) still don't have org-wide access:")
        for app_name in still_disabled:
            print(f"   - {app_name}")


## 📋 Summary

This notebook provides a complete workflow for managing organization-wide app access.

### Main Workflow (Steps 1-7):
1. ✅ Scan all apps in the workspace
2. ✅ Identify apps with "anyone in my organization can use" enabled  
3. ✅ Disable org-wide access (if `CONFIRM_DISABLE = True`)
4. ✅ Verify changes

### Optional Revert:
- **Option A**: Revert specific apps by providing a list
- **Option B**: Revert all previously disabled apps

### What Changes When Disabling:
- Removes "account users" group from app ACL
- Changes to "only people with access can use"
- Only explicitly permitted users can access

### What Changes When Reverting:
- Re-adds "account users" with CAN_USE permission
- Changes back to "anyone in my organization can use"
- All org users can access again

### What's Always Preserved:
- All other user/group/service principal permissions
- App functionality (only access control changes)
